# Use external dataset

This notebook use the dataset from the previous competitions.https://www.kaggle.com/c/petfinder-adoption-prediction

Thanks for Chris Deotte to share the idea. https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/278925

Thanks for ABHISHEK THAKUR to share the notebook baseline. https://www.kaggle.com/abhishek


# My results:
     I got bad score by doing this way. Hope anyone can make use of it and share me the way how to get a good score. XD

# import the package

In [ ]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

In [ ]:
import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import os

In [ ]:
class args:
    batch_size = 16
    image_size = 224
    epochs = 10
    fold = 9

# prepare the dataset and the model

In [ ]:
class PawpularDataset:
    def __init__(self, image_paths, targets, augmentations):
        self.image_paths = image_paths
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        try:
            image = cv2.imread(self.image_paths[item])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            print(self.image_paths[item])
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        targets = self.targets[item]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

In [ ]:
class PawpularModel(tez.Model):
    def __init__(self):
        super().__init__()

        self.model = timm.create_model("tf_efficientnet_b0_ns", pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128, 64)
        self.dense2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

        
        self.step_scheduler_after = "epoch"

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        rmse = metrics.mean_squared_error(targets, outputs, squared=False)
        return {"rmse": rmse}

    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-4, last_epoch=-1
        )
        return sch

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt

    def forward(self, image, targets=None):

        x = self.model(image)
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.sigmoid(x)
        
        if targets is not None:
            loss = nn.MSELoss()(x, targets.view(-1, 1))
            metrics = self.monitor_metrics(x, targets)
            return x, loss, metrics
        return x, 0, {}

# augmentation

In [ ]:
train_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.HorizontalFlip(p = 0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)


valid_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

# load the dataset

In [ ]:
df = pd.read_csv("../input/fake-fold-10/train_10folds.csv")
df['Pawpularity'] = df['Pawpularity'] / 100
vaccum = []
for path in df["PetID"].values:
    if os.path.exists(f"../input/petfinder-adoption-prediction/train_images/{path}-1.jpg") == False:
        vaccum.append(path)

del_index = df[df['PetID'].isin(vaccum)].index
df = df.drop(del_index)
df_train = df[df.kfold != args.fold].reset_index(drop=True)
df_valid = df[df.kfold == args.fold].reset_index(drop=True)

In [ ]:
for fold in range(10):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    train_img_paths = [f"../input/petfinder-adoption-prediction/train_images/{x}-1.jpg" for x in df_train["PetID"].values]
    valid_img_paths = [f"../input/petfinder-adoption-prediction/train_images/{x}-1.jpg" for x in df_valid["PetID"].values]
    train_dataset = PawpularDataset(
    image_paths=train_img_paths,
    targets=df_train.Pawpularity.values,
    augmentations=train_aug,
    )

    valid_dataset = PawpularDataset(
        image_paths=valid_img_paths,
        targets=df_valid.Pawpularity.values,
        augmentations=valid_aug,
    )
    model = PawpularModel()
    es = EarlyStopping(
        monitor="valid_rmse",
        model_path=f"model_f{fold}.bin",
        patience=10,
        mode="min",
        save_weights_only=True,
    )

    model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs=args.batch_size,
        valid_bs=2*args.batch_size,
        device="cuda",
        epochs=args.epochs,
        callbacks=[es],
        fp16=True,
    )